# 使用 GPU 做运算

TensorFlow 可以通过 `tf.device` 函数来指定运行每一个操作的设备。

In [2]:
import tensorflow as tf

a = tf.constant([1.,2., 3.], shape= [3], name= 'a')
b = tf.constant([1., 2., 3.],shape= [3], name= 'b')
c = a + b

通过 `log_device_placement` 参数来输出运行每个运算的设备。

In [5]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement= True))
print(sess.run(c))
sess.close()

[2. 4. 6.]


通过 `tf.device` 将运算指定到特定的设备上。

In [3]:
import tensorflow as tf

with tf.device('/cpu:0'):
    a = tf.constant([1.0, 2.0, 3.0], shape=[3], name='a')
    b = tf.constant([1.0, 2.0, 3.0], shape=[3], name='b')
with tf.device('/gpu:0'):
    c = a + b

In [4]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
print(sess.run(c))

[2. 4. 6.]


In [5]:
a_cpu = tf.Variable(0, name="a_cpu")
with tf.device('/gpu:0'):
    a_gpu = tf.Variable(0, name="a_gpu")
    # 通过 allow_soft_placement 参数自动将无法放在 GPU 上的操作放回 CPU 上。
sess = tf.Session(config=tf.ConfigProto(
    allow_soft_placement=True, log_device_placement=True))
sess.run(tf.global_variables_initializer())

`allow_soft_placement=True` 在下面的任意一个条件成立，GPU 的运算可以放到 CPU 上进行：
- 运算无法在GPU上执行；
- 没有GPU资源(比如运算被指定在第二个GPU上运行，但是机器只有一个GPU)；
- 运算输入包含对CPU计算结果的引用。

`log_device_placement=True`日志中将会记录每个节点被安排在了哪个设备上以方便调试。

虽然 GPU 可以加速 TensorFlow 的计算，但一般不会把所有的操作全部放在 GPU 上。一个比较好的实践是将计算密集型的运算放在 GPU 上，而其他操作放在 CPU 上。GPU 是相对独立的资源，将计算放入或转出 GPU 都需要额外的时间。而且 GPU 需要将计算时用到的数据从内存复制到 GPU　设备上，也需要额外的时间。因此，为了提高程序的运行速度，用户需要将相关的运算放在同一个设备上。

# 分布式运算

In [ ]:
import tensorflow as tf

# 创建一个本地集群。
s = tf.constant("Hello, distributed TensorFlow!")
server = tf.train.Server.create_local_server()
sess = tf.Session(server.target)
print(sess.run(s))

In [ ]:
# 创建两个集群
c = tf.constant("Hello from server1!")
cluster = tf.train.ClusterSpec({"local": ["localhost:2222", "localhost:2223"]})
server = tf.train.Server(cluster, job_name="local", task_index=0)
sess = tf.Session(server.target, config=tf.ConfigProto(
    log_device_placement=True))
print(sess.run(c))
server.join()


import tensorflow as tf
c = tf.constant("Hello from server2!")
cluster = tf.train.ClusterSpec({"local": ["localhost:2222", "localhost:2223"]})
server = tf.train.Server(cluster, job_name="local", task_index=1)
sess = tf.Session(server.target, config=tf.ConfigProto(
    log_device_placement=True))
print(sess.run(c))
server.join()